In [ ]:
import pandas as pd
import hvplot.pandas
import holoviews as hv
import numpy as np
from holoviews import opts
import panel as pn
pn.extension()
hv.extension('bokeh')
import geopandas as gpd
from shapely.geometry import Point
import folium
import branca.colormap as cm
import os
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from jupyter_dash import JupyterDash
from dash import Dash, dcc, html
from dash.dependencies import Input, Output
from IPython.core.display import HTML
from sklearn.linear_model import LinearRegression


In [ ]:
data_csv_file="data/commute/CSVLog_20241114_060356.CSV"
output_folder_path = 'output/date/analysis_20241114_am/'  # Change this to your folder path
#create folder folder if it does not exist
if not os.path.exists(output_folder_path):
    os.makedirs(output_folder_path)

In [ ]:
def filter(df,column,min,max): 
    mask = (df[f'{column}'] >= min) & (df[f'{column}'] < max)
    df = df[mask]
    return df

In [ ]:
def create_map(df,column,output_folder_path,output_file_name):
    df_map=df.copy()
    map_center = [44.980164, -93.185692]
    map = folium.Map(location=map_center, zoom_start=11)
    coordinates = df_map[['lat', 'long', 'MPH']].values.tolist()
    sorted_coordinates = sorted(coordinates, key=lambda x: x[2])
    colormap = cm.LinearColormap(
        colors=['blue', 'green', 'yellow'],  # Custom color range
        vmin=df_map[column].min(),
        vmax=df_map[column].max(),
        caption=column   
    )

    colormap.add_to(map)



    for i in range(len(df_map) - 1):
        # Coordinates of the current segment
        start = [df_map.iloc[i]['lat'], df_map.iloc[i]['long']]
        end = [df_map.iloc[i + 1]['lat'], df_map.iloc[i + 1]['long']]

        # Value for color mapping (use average or a specific value)
        segment_value = (df_map.iloc[i][column] + df_map.iloc[i + 1][column]) / 2

        # Create and add a PolyLine for each segment
        folium.PolyLine(
            locations=[start, end],
            color=colormap(segment_value),  # Map value to color
            weight=5,
            opacity=1
            ).add_to(map)

    # Save the map to an HTML file
    map_moving_file_path = os.path.join(output_folder_path, f'{output_file_name}.html')
    map.save(map_moving_file_path)
    return map

In [ ]:
def clean_data(df):
    # Round columns 'Time (sec)', ' Engine RPM (RPM)' and 2 other columns (Number of decimals: 0)
    df = df.round({
                'RPM': 0,
                'Engine Power (hp)': 0,
                'fuelRailPressA CMD (inHg)':0,
                'fuelRailPressA (inHg)':0,
                'Altitude (ft)':0,
                'mpgInstant': 1,
                'MPH': 1,
                'relAccPedPOS (%)': 1,
                'mpgTrip': 1,
                'Boost (psi)':1,
                'inManAbsPress (inHg)': 2,
                'massAirFlowRate (lb/min)': 2,
                'Bearing (deg)': 2,
                'accel (ft/s²)': 2,
                'accelAvg (ft/s²)': 2,
                'Time (sec)': 2,
                'distTrip (Miles)': 2,
                'barPress (inHg)':1,
                'alt (ft)':1,
                'HP':1
                })
    return df



In [ ]:
df=pd.read_csv(data_csv_file)
df.columns = df.columns.str.strip()
column_list=df.columns.tolist()
#df_columns = pd.DataFrame(column_list, columns=['ColumnName'])
#df_columns.head(20)
formatted_columns = [f'"{column}",' for column in column_list]

# Join them together with newlines
formatted_string = '\n'.join(formatted_columns)

print(formatted_string)

In [ ]:
df_clean = clean_data(df.copy())
df_clean.head()

In [ ]:
current_column_names=[
       'Trip Distance (miles)', 
       'Vehicle speed (MPH)',
       'Relative accelerator pedal position (%)',
       "Relative throttle position (%)",
       'Acceleration (ft/s²)',
       'Acceleration (Avg) (ft/s²)',
       'Engine Power (hp)', 
       'Engine RPM (RPM)',
       'Mass air flow rate (lb/min)',
       'Intake manifold absolute pressure (inHg)',
       'Commanded fuel rail pressure A (inHg)',
       'Fuel rail pressure A (inHg)',
       'Ambient air temperature (°F)',
       'Intake air temperature bank 1  sensor 1 (°F)',
       'Intake air temperature bank 1  sensor 2 (°F)',
       'Instant fuel economy (MPG)',
       'Trip Fuel Economy (MPG)',
       'Barometric pressure (inHg)',
       'Fuel type',
       'Hard Brake Count',
       'Calculated load value (%)',
       'Engine coolant temperature (°F)',
       'Catalyst temperature (Bank 1 Sensor 1) (°F)',
       'Engine oil temperature (°F)',
       'Trip Fuel (gal)',
       'Average Trip CO2 Rate (lb/mile)',
       'Engine Torque (lb•ft)',
       'Altitude (ft)',
       'Latitude (deg)',
       'Longitude (deg)',
       'Bearing (deg)',
       'PID refresh rate (Hz)',
       ]
new_column_names = [
       'distTrip (Miles)',
       'MPH',
       'relAccPedPOS (%)',
       'relThrotPOS (%)',
       'accel (ft/s²)',
       'accelAvg (ft/s²)',
       'HP',
       'RPM',
       'massAirFlowRate (lb/min)',
       'inManAbsPress (inHg)',
       'fuelRailPressA CMD (inHg)',
       'fuelRailPressA (inHg)',
       'tempAmbAir (°F)',
       'tempInBnk1Sens1 (°F)',
       'tempInBnk1Sens2 (°F)',
       'mpgInstant',
       'mpgTrip',
       'barPress (inHg)',
       'fuelType',
       'brkHardCnt',
       'loadValCalc (%)',
       'tempCool (°F)',
       'tempCatBnk1Sens1 (°F)',
       'tempOil (°F)',
       'fuelTrip (gal)',
       'avgTripCO2Rate (lb/mile)',
       'Torque (lb•ft)',
       'alt (ft)',       
       'lat',
       'long',
       'Bearing (deg)',
       'upRate (Hz)',
       ]

rename_dict = dict(zip(current_column_names, new_column_names))
df.head()

In [ ]:
df = df.rename(columns=rename_dict)
df.head()

In [ ]:
column_order = [
       'Time (sec)',
       'distTrip (Miles)',
       'MPH',
       'relAccPedPOS (%)',
       'relThrotPOS (%)',
       'accel (ft/s²)',
       'accelAvg (ft/s²)',
       'HP',
       'Torque (lb•ft)',
       'RPM',
       'Boost (psi)',
       'massAirFlowRate (lb/min)',
       'inManAbsPress (inHg)',
       'fuelRailPressA CMD (inHg)',
       'fuelRailPressA (inHg)',
       'fuelRailPressA Diff',
       'tempAmbAir (°F)',
       'tempInBnk1Sens1 (°F)',
       'tempInBnk1Sens2 (°F)',
       'tempCool (°F)',
       'tempCatBnk1Sens1 (°F)',
       'tempOil (°F)',
       'mpgInstant',
       'mpgTrip',
       'fuelType',
       'brkHardCnt',
       'loadValCalc (%)',
       'fuelTrip (gal)',
       'avgTripCO2Rate (lb/mile)',
       'barPress (inHg)',
       'alt (ft)',
       'lat',
       'long',
       'Bear (deg)',
       'reRate (Hz)',

       ]

In [ ]:
df['fuelRailPressA Diff']= df['fuelRailPressA (inHg)']-df['fuelRailPressA CMD (inHg)']


In [ ]:
existing_columns = [col for col in column_order if col in df.columns]
df = df[existing_columns]


In [ ]:
# To avoid math errors in situations where values = 0
df['Boost (psi)'] = df['Boost (psi)'].bfill().infer_objects()
df.loc[df['Boost (psi)'] == 0, 'Boost (psi)'] = 0.01

df.head(2)


In [ ]:
df_clean = clean_data(df.copy())
df_clean.head(2)

In [ ]:
df_clean.describe()

In [ ]:
df_clean.describe()

In [ ]:
#create dataframe where vehicle is stationary
df_stationary = df_clean[df_clean['MPH'] == 0]

In [ ]:
# Shift the 'Vehicle speed (MPH)' column
df_clean['next_mph'] = df_clean['MPH'].shift(-1)

# Create the mask
mask = (df_clean['MPH'] > 0) | ((df_clean['MPH'] == 0) & (df_clean['next_mph'] > 0))
mask = mask & df_clean['next_mph'].notna()
# Filter the DataFrame and drop the 'next_mph' column
df_moving = df_clean[mask].drop(columns=['next_mph'])
df_moving.describe()


In [ ]:
window_avg=50

In [ ]:
df_moving['fuelRailPressA Diff rolling_avg'] =      df_moving['fuelRailPressA Diff'].rolling(window=window_avg).mean()
df_moving['RPM rolling_avg'] =                      df_moving['RPM'].rolling(window=window_avg).mean()
df_moving['massAirFlowRate (lb/min) rolling_avg'] = df_moving['massAirFlowRate (lb/min)'].rolling(window=window_avg).mean()
df_moving['inManAbsPress (inHg) rolling_avg'] =     df_moving['inManAbsPress (inHg)'].rolling(window=window_avg).mean()
df_moving['relAccPedPOS (%) rolling_avg'] =         df_moving['relAccPedPOS (%)'].rolling(window=window_avg).mean()
df_moving['Boost (psi) rolling_avg'] =              df_moving['Boost (psi)'].rolling(window=window_avg).mean()
df_moving['MPH rolling_avg'] =                      df_moving['MPH'].rolling(window=window_avg).mean()
df_moving['Torque (lb•ft) rolling_avg'] =           df_moving['Torque (lb•ft)'].rolling(window=window_avg).mean()
df_moving['HP rolling_avg'] =                       df_moving['HP'].rolling(window=window_avg).mean()
df_moving=df_moving.dropna()
df_moving.head()


In [ ]:
plot_rolling_avg = df_moving.hvplot.line(
    x='Time (sec)',
    y='Torque (lb•ft) rolling_avg',
    color='blue',
    legend='bottom_right'
)

plot_original = df_moving.hvplot.line(
    x='Time (sec)',
    y='Torque (lb•ft)',
    color='gray',
    alpha=0.4,
    legend='bottom_right'
)

# Overlay the two plots
combined_plot = plot_original * plot_rolling_avg
combined_plot

In [ ]:
plot_rolling_avg = df_moving.hvplot.line(
    x='Time (sec)',
    y='HP rolling_avg',
    color='blue',
    legend='bottom_right'
)

plot_original = df_moving.hvplot.line(
    x='Time (sec)',
    y='HP',
    color='gray',
    alpha=0.4,
    legend='bottom_right'
)

# Overlay the two plots
combined_plot = plot_original * plot_rolling_avg
combined_plot

In [ ]:
plot_rolling_avg = df_moving.hvplot.line(
    x='Time (sec)',
    y='MPH rolling_avg',
    color='blue',
    legend='bottom_right'
)

plot_original = df_moving.hvplot.line(
    x='Time (sec)',
    y='MPH',
    color='gray',
    alpha=0.4,
    legend='bottom_right'
)

# Overlay the two plots
combined_plot = plot_original * plot_rolling_avg
combined_plot

In [ ]:
plot_rolling_avg = df_moving.hvplot.line(
    x='Time (sec)',
    y='Boost (psi) rolling_avg',
    color='blue',
    legend='bottom_right'
)

plot_original = df_moving.hvplot.line(
    x='Time (sec)',
    y='Boost (psi)',
    color='gray',
    alpha=0.4,
    legend='bottom_right'
)

# Overlay the two plots
combined_plot = plot_original * plot_rolling_avg
combined_plot

In [ ]:
plot_rolling_avg = df_moving.hvplot.line(
    x='Time (sec)',
    y='relAccPedPOS (%) rolling_avg',
    color='blue',
    legend='bottom_right'
)

plot_original = df_moving.hvplot.line(
    x='Time (sec)',
    y='relAccPedPOS (%)',
    color='gray',
    alpha=0.4,
    legend='bottom_right'
)

# Overlay the two plots
combined_plot = plot_original * plot_rolling_avg
combined_plot


In [ ]:
plot_rolling_avg = df_moving.hvplot.line(
    x='Time (sec)',
    y='inManAbsPress (inHg) rolling_avg',
    color='blue',
    legend='bottom_right'
)

plot_original = df_moving.hvplot.line(
    x='Time (sec)',
    y='inManAbsPress (inHg)',
    color='gray',
    alpha=0.4,
    legend='bottom_right'
)

# Overlay the two plots
combined_plot = plot_original * plot_rolling_avg
combined_plot

In [ ]:
plot_rolling_avg = df_moving.hvplot.line(
    x='Time (sec)',
    y='massAirFlowRate (lb/min) rolling_avg',
    color='blue',
    legend='bottom_right'
)

plot_original = df_moving.hvplot.line(
    x='Time (sec)',
    y='massAirFlowRate (lb/min)',
    color='gray',
    alpha=0.4,
    legend='bottom_right'
)

# Overlay the two plots
combined_plot = plot_original * plot_rolling_avg
combined_plot

In [ ]:
plot_rolling_avg = df_moving.hvplot.line(
    x='Time (sec)',
    y='RPM rolling_avg',
    color='blue',
    legend='bottom_right'
)

plot_original = df_moving.hvplot.line(
    x='Time (sec)',
    y='RPM',
    color='gray',
    alpha=0.4,
    legend='bottom_right'
)

# Overlay the two plots
combined_plot = plot_original * plot_rolling_avg
combined_plot

In [ ]:

plot = df_moving.hvplot(
    kind='line',
    x='Time (sec)',           # x-axis column
    y='fuelRailPressA Diff rolling_avg',   # y-axis column
    legend='bottom_right',  # Position the legend
    widget_location='bottom' # Widget for interaction at the bottom
    )
plot



In [ ]:
plot = df_moving.hvplot(
    kind='line',
    x='distTrip (Miles)',   # X-axis column
    y='alt (ft)',           # Y-axis column
    legend='bottom_right',  # Position the legend
    widget_location='bottom' # Widget for interaction at the bottom
    )
plot

In [ ]:
df_0_10=filter(df_moving,'MPH',0,10)
df_0_10.describe()


In [ ]:
hvexplorer = df_0_10.hvplot.explorer()
hvexplorer

In [ ]:
df_10_20=filter(df_moving,'MPH',10,20)
df_10_20.describe()

In [ ]:
hvexplorer = df_10_20.hvplot.explorer()
hvexplorer

In [ ]:
df_20_30=filter(df_moving,'MPH',20,30)
df_20_30.describe()


In [ ]:
hvexplorer = df_20_30.hvplot.explorer()
hvexplorer


In [ ]:
df_30_40=filter(df_moving,'MPH',30,40)
df_30_40.describe()

In [ ]:
hvexplorer = df_30_40.hvplot.explorer()
hvexplorer

In [ ]:
df_40_50=filter(df_moving,'MPH',40,50)
df_40_50.describe()

In [ ]:
hvexplorer = df_40_50.hvplot.explorer()
hvexplorer

In [ ]:
df_50_60=filter(df_moving,'MPH',50,60)
df_50_60.describe()

In [ ]:
hvexplorer = df_50_60.hvplot.explorer()
hvexplorer

In [ ]:
df_60_70=filter(df_moving,'MPH',60,70)
df_60_70.describe()

In [ ]:
hvexplorer = df_60_70.hvplot.explorer()
hvexplorer

In [ ]:
df_70_85=filter(df_moving,'MPH',70,85)
df_70_85.describe()

In [ ]:
hvexplorer = df_70_85.hvplot.explorer()
hvexplorer

In [ ]:
df_55_80=filter(df_moving,'MPH',55,80)
df_55_80.describe()

In [ ]:
hvexplorer = df_55_80.hvplot.explorer()
hvexplorer

In [ ]:
map_55_80 = create_map(df_55_80,'MPH rolling_avg',output_folder_path,"map_55_80")
map_55_80

In [ ]:
map_55_80 = create_map(df_55_80,'alt (ft)',output_folder_path,"map_55_80")
map_55_80

In [ ]:
if 'fuelRailPressA CMD (inHg)' in df_moving.columns:
    df_fuel_rail = df_55_80[['RPM','fuelRailPressA CMD (inHg)', 'fuelRailPressA (inHg)']].copy()
    df_fuel_rail['cmd/act'] = df_fuel_rail['fuelRailPressA CMD (inHg)'] / df_fuel_rail['fuelRailPressA (inHg)'].replace(0, np.nan)
    df_fuel_rail['cmd-act'] = df_fuel_rail['fuelRailPressA CMD (inHg)'] - df_fuel_rail['fuelRailPressA (inHg)'].replace(0, np.nan)
    df_fuel_rail.describe()

In [ ]:
hvexplorer = df_fuel_rail.hvplot.explorer()
hvexplorer

In [ ]:
hvexplorer = df_55_80.hvplot.explorer()
hvexplorer

In [ ]:
# Independent variables
X = df_moving[['RPM rolling_avg', 'massAirFlowRate (lb/min) rolling_avg']]
#print(X.shape)
# Dependent variable
y = df_moving['HP rolling_avg']

# Create and train the model
model = LinearRegression()
model.fit(X, y)

# Make predictions
df_moving['predicted_hp'] = model.predict(X)

# Plotting the actual vs predicted prices using hvplot
plot = df_moving.hvplot.scatter(
    x='Time (sec)', 
    y='HP rolling_avg',
    color='blue',
    label='Actual HP'
) * df_moving.hvplot.line(
    x='Time (sec)',
    y='predicted_hp',
    color='red',
    label='Predicted HP'
)

# Set plot size, title, labels, etc.
plot = plot.opts(
    title="Actual vs Predicted HP", 
    xlabel="Square RPM", 
    ylabel="HP",
    width=1200,  # Adjust width as needed
    height=600  # Adjust height as needed
)

plot



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming df_moving is already defined and contains the necessary data

# Independent variables
X = df_moving[['RPM rolling_avg', 'massAirFlowRate (lb/min) rolling_avg']]
# Dependent variable
y = df_moving['HP rolling_avg']

# Create and train the model
model = LinearRegression()
model.fit(X, y)

# Make predictions
df_moving['predicted_hp'] = model.predict(X)

# Plotting the actual vs predicted prices using matplotlib
plt.figure(figsize=(12, 6))

# Scatter plot for actual HP
plt.scatter(df_moving['Time (sec)'], df_moving['HP rolling_avg'], color='blue', label='Actual HP')

# Line plot for predicted HP
plt.plot(df_moving['Time (sec)'], df_moving['predicted_hp'], color='red', label='Predicted HP')

# Set plot size, title, labels, etc.
plt.title("Actual vs Predicted HP")
plt.xlabel("Time (sec)")
plt.ylabel("HP")
plt.legend()
plt.show()


In [ ]:
# Plotting HP vs RPM rolling_avg
plt.figure(figsize=(12, 6))
plt.scatter(df_moving['RPM rolling_avg'], df_moving['HP rolling_avg'], color='blue', label='Actual HP')
plt.plot(df_moving['RPM rolling_avg'], model.predict(X), color='red', label='Predicted HP')
plt.title("HP vs RPM rolling_avg")
plt.xlabel("RPM rolling_avg")
plt.ylabel("HP")
plt.legend()
plt.show()

# Plotting HP vs massAirFlowRate (lb/min) rolling_avg
plt.figure(figsize=(12, 6))
plt.scatter(df_moving['massAirFlowRate (lb/min) rolling_avg'], df_moving['HP rolling_avg'], color='blue', label='Actual HP')
plt.plot(df_moving['massAirFlowRate (lb/min) rolling_avg'], model.predict(X), color='red', label='Predicted HP')
plt.title("HP vs massAirFlowRate (lb/min) rolling_avg")
plt.xlabel("massAirFlowRate (lb/min) rolling_avg")
plt.ylabel("HP")
plt.legend()
plt.show()


In [ ]:
hvexplorer=df_predict_hp.hvplot.explorer()
hvexplorer